In his PhD thesis on GA optimisation of trading Lidquist uses the following strategy:

![Lindquist-gene-encoding](images/Lindquist-gene-encoding01.png)
![Lindquist-gene-encoding](images/Lindquist-gene-encoding02.png)
![Lindquist-gene-encoding](images/Lindquist-gene-encoding03.png)
![Lindquist-gene-encoding](images/Lindquist-gene-encoding04.png)



Corne and Lones writes in 'Handbook of Heuristics', Chapter Evolutionary Aalgorithms, Springer, 2018:

In common with other optimisation algorithms, most EAs are designed to work with and optimise
vectors (or, equivalently, lists or arrays) of decision variables. Solutions to many kinds of problems
can be represented, either directly or indirectly, in this form. However, EAs are not limited to working
with vectors, and there are often advantages to working directly with representations that are more
natural for the problem domain: for example, matrices, trees, graphs, rule sets, etc. The
general approach is the same as for the EAs discussed in the previous section, except that specialised
initialisation routines and variation operators are used to randomly create, mutate and recombine
instances of the appropriate solution representation.

In [5]:
import matplotlib.pyplot as plt
import sys
import array
import random
import numpy as np

from deap import  algorithms
from deap import  base
from deap import  creator
from deap import  tools

%matplotlib inline


In [47]:
POP_SIZE = 100
MAX_GEN = 100
MUT_PROB = 0.2
CX_PROB = 0.8

timemax = 5


In [48]:
from enum import Enum,unique



def round_down(x, a):
    return round(x / a) * a

# Only these are allowed as Ftypes
@unique
class Ftype(Enum):
    M = 0
    S = 1
    
# Only these are allowed as Dirtype
@unique
class Dirtype(Enum):
    P = 0
    S = 1
    
def generate_event():
    di = {}
    # time between 0 and timemax
    di['time'] = round_down(random.uniform(0, timemax),0.05)
    # even odds for each of the two cases
    di['Ftype'] = Ftype.S if random.uniform(0, 1) <=0.5 else Ftype.M
    # even odds for each of the two cases
    di['DirType'] = Dirtype.P if random.uniform(0, 1) <=0.5 else Dirtype.S
    return di


The objective is to minimise two parameters:
- the difference between the required code and the individual code
- the number of events


In [49]:
creator.create("FitnessMin", base.Fitness,weights=(-1.0,-1.0))
creator.create("Individual",dict, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual,generate_event) 
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

pop = toolbox.population(n=POP_SIZE)


In [50]:
print(f'Individual: {creator.Individual()}')  
print(f'individual: {toolbox.individual()}')  
print(f'population: {toolbox.population(n=10)}') 

Individual: {}
individual: {'time': 4.3, 'Ftype': <Ftype.M: 0>, 'DirType': <Dirtype.S: 1>}
population: [{'time': 2.0, 'Ftype': <Ftype.M: 0>, 'DirType': <Dirtype.S: 1>}, {'time': 0.8, 'Ftype': <Ftype.S: 1>, 'DirType': <Dirtype.S: 1>}, {'time': 0.9, 'Ftype': <Ftype.S: 1>, 'DirType': <Dirtype.S: 1>}, {'time': 1.6500000000000001, 'Ftype': <Ftype.S: 1>, 'DirType': <Dirtype.P: 0>}, {'time': 2.6500000000000004, 'Ftype': <Ftype.M: 0>, 'DirType': <Dirtype.S: 1>}, {'time': 2.6, 'Ftype': <Ftype.M: 0>, 'DirType': <Dirtype.P: 0>}, {'time': 3.9000000000000004, 'Ftype': <Ftype.M: 0>, 'DirType': <Dirtype.S: 1>}, {'time': 4.1000000000000005, 'Ftype': <Ftype.S: 1>, 'DirType': <Dirtype.P: 0>}, {'time': 1.4000000000000001, 'Ftype': <Ftype.M: 0>, 'DirType': <Dirtype.P: 0>}, {'time': 4.95, 'Ftype': <Ftype.M: 0>, 'DirType': <Dirtype.S: 1>}]


In [51]:
toolbox.register('mate',tools.cxOrdered)
toolbox.register('mutate',tools.mutShuffleIndexes, indpb=0.05)
toolbox.register('select',tools.selTournament, tournsize=3)


In [52]:
ind = toolbox.individual()

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [53]:
def evalTSP(individual):
    diffx = np.diff(x[individual])
    diffy = np.diff(y[individual])
    distance = np.sum(diffx**2 + diffy**2)
    return (distance,)

# register the evaluation function with the toolbox
toolbox.register('evaluate',evalTSP)

In [54]:
#initial populatiom
pop = toolbox.population(n=POP_SIZE)

# save only the best individual
hof = tools.HallOfFame(1)

# code the GA algorithm
result, log = algorithms.eaSimple(pop,toolbox,cxpb=CX_PROB, mutpb=MUT_PROB, stats=stats,
                                  ngen=MAX_GEN, halloffame=hof,verbose=True)



NameError: name 'x' is not defined